In [ ]:
import numpy as np
import contextlib
import os

import rubin_sim.maf as maf

from u_band_strat import data_dir, interpolate_number_density

In [33]:
# Collect data for the table of median depths
depths = dict()
for scale in [1.0, 1.1, 1.2, 1.5]:
    depths[scale] = dict()
    for expt in [30, 38, 45, 60]:
        if expt == 60 and scale == 1.5:
            continue
        depths[scale][expt] = dict()
        for band in "ugrizy":
            depths[scale][expt][band] = dict()
            for year in [1, 10]:
                # Load the dropout band map
                scale_str = str(scale).replace(".", "_")
                with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
                    m5 = maf.MetricBundle.load(
                        data_dir
                        / "m5_maps"
                        / f"internal_u_expt{expt}_nscale{scale_str}v3_4_{year}yrs_ExgalM5_{band}.npz",
                    ).metric_values
                depths[scale][expt][band][year] = np.ma.median(m5)

In [ ]:
for scale in depths:
    for expt in depths[scale]:
        row = f"{scale:.1f}x, {expt}s $u$ & {scale * expt / 30:.2f} & "
        for band in "ugrizy":
            row += f"{depths[scale][expt][band][1] - depths[1.0][30][band][1]:>5.2f} "
            row += f"{f'({depths[scale][expt][band][10] - depths[1.0][30][band][10]:.2f}':>6}) & "
        row = row[:-2] + r"\\"
        print(row)